In [105]:
import langchain

from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.agents import AgentType, load_tools, initialize_agent
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, ConversationChain

load_dotenv()

True

### 1. Langchain Basic

In [6]:
client = OpenAI()


In [7]:
prompt = "Can you tell me the total number of countries in Asia? Can you name the top 10 based on population"
print(client.predict(prompt))



As of 2021, there are 48 countries in Asia. The top 10 countries in Asia based on population are:

1. China
2. India
3. Indonesia
4. Pakistan
5. Bangladesh
6. Japan
7. Philippines
8. Vietnam
9. Iran
10. Turkey


### 2. Prompt Template

In [9]:
# 1st way 
prompt_template_name = PromptTemplate(
    input_variables = ['country'],
    template = 'Can you tell me the capital of {country}?'
)

In [10]:
prompt1 = prompt_template_name.format(country = 'India')
prompt2 = prompt_template_name.format(country = 'Romania')

In [12]:
client.predict(prompt1).strip()

'The capital of India is New Delhi.'

In [13]:
client.predict(prompt2).strip()

'The capital of Romania is Bucharest.'

In [14]:
# 2nd way
prompt_template_name_2 = PromptTemplate.from_template("What is a good name for a company that makes {products}?")

In [15]:
prompt3 = prompt_template_name_2.format(products = 'toys')

In [17]:
print(client.predict(prompt3).strip())

1. Playful Creations Co.
2. Toytopia Inc.
3. WonderWorks Toys
4. Imagination Factory
5. Funland Toys
6. Happyland Playthings
7. Toylandia Enterprises
8. Creativity Kids Co.
9. Joyful Jax Toys
10. Whimsy World Inc.
11. Dreamland Toys
12. Magic Makers Co.
13. The Toy Emporium
14. Playtime Innovations
15. Toy Kingdom Co.
16. Imagine and Play Co.
17. Sunny Side Toys
18. SparklePlay Co.
19. Toytopolis Inc.
20. WonderWorks Toy Co.


### 3. Agent

In [18]:
# Let's first see what's the problem

prompt4 = "Can you tell me who won the recent football world cup?"
prompt5 = "Can you tell me the current GDP of India?"

In [19]:
print(client.predict(prompt4).strip())

The recent football world cup, also known as the 2018 FIFA World Cup, was won by France. They defeated Croatia 4-2 in the final match.


In [20]:
print(client.predict(prompt5).strip())

As of 2021, the current GDP of India is approximately $3.05 trillion USD.


The information we get is dependent on the last trained data, and hence, is not recent. In order to overcome the problem, we use Agents who can scrape the data
from the internet using APIs.
For extracting real-time information, we will be using SerpAI ( pip install google-search-results)

In [34]:
tool = load_tools(["serpapi"], llm=client)
agent = initialize_agent(tool, client, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [35]:
agent.run("Can you tell me who won the recent football world cup?")



> Entering new AgentExecutor chain...
 I should try searching for the answer.
Action: Search
Action Input: "who won the recent football world cup"
Observation: {'title': '2022 World Cup', 'games': [{'tournament': 'World Cup', 'stage': 'Final', 'stadium': 'Lusail Stadium', 'stadium_kgmid': '/m/0fq1625', 'status': 'FT (P)', 'date': 'Dec 18, 22', 'video_highlights': {'link': 'https://www.plus.fifa.com/content/46772b92-24c3-4315-87da-158ef7af6259?cid=(p_go-box)(c_FIFAPlus)(sc_highlights)(ssc_)(da_)(co_)(cc_)&s_cid=(co_)(cc_)&c=(c_FIFAPlus)(sc_highlights)(ssc_)(cc_)&pid=(p_go-box)(ch_)(t_)', 'thumbnail': 'https://serpapi.com/searches/668ec377ef7dda8ba5c1e252/images/405aab89c4c1d87d5cc981426bdd93ced1fd99cf3b9d23cce91be909e40f404f53e0014f444dd676ceb9c91be4f28304b741d23dc93aba10.jpeg', 'duration': '2:0'}, 'teams': [{'name': 'Argentina', 'score': '3 (4)', 'kgmid': '/m/02bh_v', 'thumbnail': 'https://serpapi.com/searches/668ec377ef7dda8ba5c1e252/images/405aab89c4c1d87d5cc981426bdd93cea6fe1a12c9

'Argentina national football team'

In [36]:
agent.run("Can you tell me the current GDP of India?")



> Entering new AgentExecutor chain...
 I should think about using a search engine to find this information
Action: Search
Action Input: "Current GDP India"
Observation: ["India, officially the Republic of India, is a country in South Asia. It is the seventh-largest country by area; the most populous country as of June 2023; and from the time of its independence in 1947, the world's most populous democracy.", 'India type: Country in South Asia.', 'India entity_type: countries, locations, travel, competitions, military, travel.', 'India kgmid: /m/03rk0.', 'India capital: New Delhi.', 'India co2_emissions_per_capita: 1.58 metric tons (2020).', 'India electricity_consumption_per_capita: 797.35 kWh (2014).', 'India energy_use_per_capita: 630.90 kg of oil equivalent (2014).', 'India fertility_rate: 2.03 births per woman (2021).', 'India gdp_growth_rate: 7.2% annual change (2022).', 'India gross_domestic_product: 3.417 trillion USD (2022).', 'Economy of India ; Increase $3.937 trillion (nom

'The current GDP of India is approximately $3.94 trillion USD (2023).'

In [38]:
# Let's create another tool with wikipedia(!pip install wikipedia)
wiki_tool = load_tools(['wikipedia'], llm=client)
wiki_agent = initialize_agent(wiki_tool, client, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [39]:
wiki_agent.run("Can you tell me anything about Euro 2024?")



> Entering new AgentExecutor chain...
 It would be helpful to check Wikipedia for information on Euro 2024.
Action: wikipedia
Action Input: Euro 2024
Observation: Page: UEFA Euro 2024
Summary: The 2024 UEFA European Football Championship, commonly referred to as UEFA Euro 2024 (stylised as UEFA EURO 2024), or simply Euro 2024, is the ongoing 17th edition of the UEFA European Championship, the quadrennial international football  championship organised by UEFA for the European men's national teams of its member associations. Germany is hosting the tournament, which is taking place from 14 June to 14 July 2024. The tournament comprises 24 teams, with Georgia making their European Championship debut. 
It is the third time that European Championship matches are played on German territory and the second time in reunified Germany, as West Germany hosted the tournament's 1988 edition, and four matches of the multi-national Euro 2020 were played in Munich. It is the first time the competition

'Euro 2024 is the 17th edition of the UEFA European Championship, hosted by Germany in 2024. It comprises of 24 teams and is the first major tournament held in Germany since the 2006 FIFA World Cup. The knockout stage began on June 29, 2024 and the final will be held on July 14, 2024 at Olympiastadion in Berlin. The qualifying tournament, which took place from March 2023 to March 2024, determined the 23 teams that would join the host team.'

### 4. Chain

Central to Langchain is a vital component called Chains. This forms the core connection between one or more LLMs. In certain sophisticated applications, it becomes necessary to chain LLMs together, either with each other or with other elements.

In [41]:
client

OpenAI(client=<openai.resources.completions.Completions object at 0x7fa943253fd0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7fa942f19f30>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [43]:
chain_prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
chain_prompt

PromptTemplate(input_variables=['product'], template='What is a good name for a company that makes {product}')

In [46]:
chain = LLMChain(llm=client, prompt=chain_prompt, verbose=True)
chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['product'], template='What is a good name for a company that makes {product}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x7fa943253fd0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7fa942f19f30>, openai_api_key=SecretStr('**********'), openai_proxy=''))

In [47]:
chain.run("Wine").strip()



> Entering new LLMChain chain...
Prompt after formatting:
What is a good name for a company that makes Wine

> Finished chain.


'"Vineyard Vines" or "Grapevine Co."'

In [48]:
# Combining Multiple Chains by SimpleSequentialChain
prompt_startup_name = PromptTemplate(
    input_variables = ['startup_name'],
    template = "I want to start a startup for {startup_name}, suggest me a good name for this!"
)

name_chain = LLMChain(llm=client, prompt=prompt_startup_name)

prompt_startup_strategy = PromptTemplate(
    input_variables = ['name'],
    template = "Suggest some strategies for {name}"
)

strategy_chain = LLMChain(llm=client, prompt=prompt_startup_strategy)

In [52]:
chain = SimpleSequentialChain(chains=[name_chain, strategy_chain], verbose=True)
print(chain.run("Artificial Intelligence"))



> Entering new SimpleSequentialChain chain...


"IntelliGenius" 


1. Market research and targeting: Conduct thorough market research to identify the target audience for IntelliGenius. This will help in understanding their needs, preferences, and behaviors, which can then be used to develop effective marketing strategies.

2. Social media presence: Create a strong presence on social media platforms like Facebook, Twitter, and LinkedIn. This will help in reaching out to a larger audience and also engage with potential customers.

3. Influencer marketing: Collaborate with influencers in the field of education, technology, and innovation to promote IntelliGenius. This will help in building credibility and trust among the target audience.

4. Content marketing: Develop high-quality and informative content such as blogs, articles, videos, and infographics related to education and technology. This will help in establishing IntelliGenius as a thought leader in the industry.

5. Partnership 

In [55]:
# Replace SimpleSequentialChain with SequentialChain
prompt_startup_name_seq = PromptTemplate(
    input_variables = ['startup_field'],
    template = "I want to start a startup for {startup_field}, suggest me a good name for this!"
)

name_chain_seq = LLMChain(llm=client, prompt=prompt_startup_name_seq, output_key="startup_name")

prompt_startup_strategy_seq = PromptTemplate(
    input_variables = ['startup_name'],
    template = "Suggest some strategies for {startup_name}"
)

strategy_chain_seq = LLMChain(llm=client, prompt=prompt_startup_strategy_seq, output_key="strategy_items")

In [58]:
seq_chain = SequentialChain(
    chains=[name_chain_seq, strategy_chain_seq],
    input_variables=['startup_field'],
    output_variables=['startup_name', 'strategy_items'],
    verbose=True)

seq_chain({"startup_field": "Artificial Intelligence"})



> Entering new SequentialChain chain...

> Finished chain.


{'startup_field': 'Artificial Intelligence',
 'startup_name': '\n\n1. "Cognify"\n2. "AIgenius"\n3. "NeuroNexus"\n4. "MindMeld"\n5. "IntelliVerse"\n6. "Cortex Innovations"\n7. "DeepThink"\n8. "Prognosis AI"\n9. "SmartBotics"\n10. "Sentient Solutions"',
 'strategy_items': ' \n\n1. "Cognify":\na. Develop a user-friendly interface that simplifies complex cognitive tasks.\nb. Partner with educational institutions to promote learning and training through the use of Cognify.\nc. Utilize gamification techniques to make cognitive tasks more engaging and fun.\nd. Offer personalized training programs based on individual needs and goals.\ne. Collaborate with businesses to integrate Cognify into their employee training and development programs.\nf. Conduct research and publish white papers to establish credibility and showcase the benefits of using Cognify.\n\n2. "AIgenius":\na. Position AIgenius as a cutting-edge and innovative solution for businesses.\nb. Showcase case studies and success stories

### 5. PDFReader

In [76]:
loader = PyPDFLoader("BiasFairnessSurvey.pdf")
pages = loader.load_and_split()

In [87]:
print(pages[0].page_content)

A Survey on Bias and Fairness in Machine Learning
NINAREH MEHRABI, FRED MORSTATTER, NRIPSUTA SAXENA,
KRISTINA LERMAN, and ARAM GALSTYAN, USC-ISI
With the widespread use of artificial intelligence (AI) systems and applications in our everyday lives, accounting
for fairness has gained significant importance in designing and engineering of such systems. AI systems can be
used in many sensitive environments to make important and life-changing decisions; thus, it is crucial to ensure
that these decisions do not reflect discriminatory behavior toward certain groups or populations. More recently
some work has been developed in traditional machine learning and deep learning that address such challenges in
different subdomains. With the commercialization of these systems, researchers are becoming more aware of
the biases that these applications can contain and are attempting to address them. In this survey we investigated
different real-world applications that have shown biases in various ways,

### 6. Memory

In [92]:
prompt_template_non_mem = PromptTemplate(
    input_variables = ['product'],
    template = "What is a good name for a company that make {product}"
)

non_mem_chain = LLMChain(llm=client, prompt=prompt_template_non_mem)
print(non_mem_chain.run("medical devices").strip())

"VitalTech"


In [93]:
non_mem_chain.memory

In [95]:
## ConversationBufferMemory: Attach memory to all previous conversations
memory = ConversationBufferMemory()

In [96]:
prompt_template_mem = PromptTemplate(
    input_variables = ['product'],
    template = "What is a good name for a company that make {product}"
)

mem_chain = LLMChain(llm=client, prompt=prompt_template_mem, memory=memory)
print(mem_chain.run("medical devices").strip())

1. MedTech Innovations
2. Vitality Devices
3. ProMed Solutions
4. HealthTech Dynamics
5. Precision Medical Devices
6. Wellness Works
7. CareTech Innovations
8. LifeSavers Medical
9. MedEquip Co.
10. Advanced Medical Devices


In [99]:
print(mem_chain.run("camera").strip())

1. Capture Co.
2. LensCrafters
3. Focal Point
4. Shutterworks
5. SnapTech
6. Frame & Focus
7. ImageMakers
8. Aperture Enterprises
9. FlashForward
10. ZoomWorks
11. SnapShots Inc.
12. Visionary Cameras
13. Pixel Producers
14. Frame & Film Co.
15. OpticTech
16. CameraCrafters
17. SnapHive
18. LensLogic
19. FocusFusion
20. ShotMakers.


In [100]:
print(mem_chain.run("Drones").strip())

"SkyTech Drones"


In [101]:
mem_chain.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='medical devices'), AIMessage(content='\n\n1. MedTech Innovations\n2. Vitality Devices\n3. ProMed Solutions\n4. HealthTech Dynamics\n5. Precision Medical Devices\n6. Wellness Works\n7. CareTech Innovations\n8. LifeSavers Medical\n9. MedEquip Co.\n10. Advanced Medical Devices'), HumanMessage(content='camera'), AIMessage(content='\n\n1. Capture Co.\n2. LensCrafters\n3. Focal Point\n4. Shutterworks\n5. SnapTech\n6. Frame & Focus\n7. ImageMakers\n8. Aperture Enterprises\n9. FlashForward\n10. ZoomWorks\n11. SnapShots Inc.\n12. Visionary Cameras\n13. Pixel Producers\n14. Frame & Film Co.\n15. OpticTech\n16. CameraCrafters\n17. SnapHive\n18. LensLogic\n19. FocusFusion\n20. ShotMakers.'), HumanMessage(content='Drones'), AIMessage(content='\n\n"SkyTech Drones" ')]))

In [103]:
print(mem_chain.memory.buffer)

Human: medical devices
AI: 

1. MedTech Innovations
2. Vitality Devices
3. ProMed Solutions
4. HealthTech Dynamics
5. Precision Medical Devices
6. Wellness Works
7. CareTech Innovations
8. LifeSavers Medical
9. MedEquip Co.
10. Advanced Medical Devices
Human: camera
AI: 

1. Capture Co.
2. LensCrafters
3. Focal Point
4. Shutterworks
5. SnapTech
6. Frame & Focus
7. ImageMakers
8. Aperture Enterprises
9. FlashForward
10. ZoomWorks
11. SnapShots Inc.
12. Visionary Cameras
13. Pixel Producers
14. Frame & Film Co.
15. OpticTech
16. CameraCrafters
17. SnapHive
18. LensLogic
19. FocusFusion
20. ShotMakers.
Human: Drones
AI: 

"SkyTech Drones" 


In [107]:
## ConversationChain: Helps to remember last x conversation chains
convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [108]:
convo.run("Who won the first Football World Cup?")

' The first Football World Cup was won by Uruguay in 1930. They defeated Argentina 4-2 in the final match. This was the inaugural World Cup and only 13 teams participated. The final was held in the Estadio Centenario in Montevideo, Uruguay.'

In [109]:
convo.run("What is the output of 4*4+1?")

' The output of 4*4+1 is 17. This is because the multiplication of 4 and 4 equals 16, and adding 1 to that gives us a total of 17.'

In [110]:
convo.run("What is the output of 4*2/1?")

' The output of 4*2/1 is 8. This is because 4 multiplied by 2 equals 8, and then dividing that by 1 does not change the value, so the output remains 8.'

In [111]:
convo.run("Who was the captain of the winning team?")

" The captain of the Uruguay team that won the first Football World Cup in 1930 was José Nasazzi. He was a defender and also the captain of the club team Nacional in Uruguay. Nasazzi played in all four of Uruguay's matches during the World Cup and scored one goal."

In [125]:
# ConversationBufferWindowMemory: Sets a buffer of k and remembers k conversations
convoBufferMemory = ConversationBufferWindowMemory(k=1) # Change the k and rerun to get the difference

In [126]:
convo_buffer = ConversationChain(llm=OpenAI(temperature=0.7), memory=convoBufferMemory)

In [127]:
convo_buffer.run("Who won the first Football World Cup?")

' The first Football World Cup was won by Uruguay in 1930. They beat Argentina 4-2 in the final match. The tournament was held in Uruguay and was organized by FIFA. It consisted of 13 teams, and Uruguay won all four of their matches, scoring 15 goals and only conceding three. The top scorer of the tournament was Guillermo Stabile from Argentina, with eight goals. The trophy was called the Jules Rimet Trophy, named after the president of FIFA at the time.'

In [128]:
convo_buffer.run("What is the output of 4*4+1?")

' The output of 4*4+1 is 17.'

In [129]:
convo_buffer.run("Who was the captain of the winning team?")

' I do not have enough information to accurately answer that question. Do you have a specific team and game in mind?'